<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex05notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex05notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024)


----
## ロジスティック回帰による手書き数字の識別
----



最短距離法や最近傍法の実験に使ったのと同じ手書き数字のデータを使ってロジスティック回帰モデルの学習の実験をやってみましょう．
このデータがどんなものかという話は，最短距離法や最近傍法について説明した notebook の方に書いています．

---
### 準備

まずはいろいろ import．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のロジスティック回帰
from sklearn.linear_model import LogisticRegression

次はデータの準備．

In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
datL = rv['datL'].astype(float)
labL = rv['labL']
datT = rv['datT'].astype(float)
labT = rv['labT']
print(datL.shape, labL.shape, datT.shape, labT.shape)

K = 10 # クラス数

# 学習データの用意
NL, D = datL.shape # 学習データの数と次元数
XLraw = datL/255.0
XLm = np.mean(XLraw, axis=0)
XL = np.empty((NL, D+1))
XL[:, 0] = 1.0
XL[:, 1:] = XLraw - XLm
YL = np.zeros((NL, K))
for ik in range(K):
    YL[labL == ik, ik] = 1.0

# テストデータの用意
NT, _ = datT.shape # テストデータの数
XTraw = datT/255.0
XT = np.empty((NT, D+1))
XT[:, 0] = 1.0
XT[:, 1:] = XTraw - XLm
YT = np.zeros((NT, K))
for ik in range(K):
    YT[labT == ik, ik] = 1.0

次はモデル出力の計算と学習のための関数の定義です．

In [ ]:
# モデル出力の計算
def modelK(X, W):
    s = np.exp(X @ W.T)
    Yt = s / np.sum(s, axis=1)[:, np.newaxis]
    return Yt

# 交差エントロピーと正解数
def scoreK(Y, Yt):
    ce = -np.sum(Y * np.log(Yt))/len(Y)
    count = np.sum(np.argmax(Y, axis=1) == np.argmax(Yt, axis=1))
    return ce, count

# 勾配の計算
def gradK(X, Y, Yt):
    return (Yt - Y).T @ X

データを可視化する関数の定義．

In [ ]:
# データを画像として表示するための関数
#
def display(data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[0] == nx*ny
    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    # 画像の出力
    plt.axis('off')
    plt.imshow(img, cmap = 'gray')
    plt.show()

---
### 実験

以下のセルが，上記のデータを 10 クラスに分類するロジスティック回帰の学習のコードです．

In [ ]:
# パラメータの初期化
W = (np.random.random((K, D+1)) - 0.5) * 0.2 # [-0.1, 0.1) の一様乱数

# 学習係数と学習繰り返し回数
eta = 5.0/NL
nitr = 500

# 学習
for i in range(nitr+1):
    Yt = modelK(XL, W)        # モデル出力の計算
    ce, count = scoreK(YL, Yt) # 交差エントロピーと正解数の計算
    dW = gradK(XL, YL, Yt)    # 勾配の計算
    W -= eta * dW             # パラメータの更新
    if (i < 100 and i % 10 == 0) or (i % 100 == 0):
        print(f'{i}  {ce:.3f}  {count/NL:.3f}')

出力される数値は，左から順に「学習繰り返し回数」，「（学習データに対する）交差エントロピー」，「（同）識別率（識別の正答率）」です．学習を繰り返すごとに交差エントロピーの値が減少していること，それに連れて識別率が上昇していることが分かります．



次のコードセルを実行すると，学習したモデルにテストデータを入力してクラスを予測させることができます．

ロジスティック回帰では，パラメータの初期値によって学習結果が変わります．
↑のセルを実行するたびに異なる初期値で学習することになりますので，学習して↓のセルを実行することを何度かやって，結果がどう変わるか観察するとよいでしょう．


In [ ]:
# テスト
Yt = modelK(XT, W)        # モデル出力の計算
ce, count = scoreK(YT, Yt) # 交差エントロピーと正解数の計算

# 一部の結果の可視化
nx, ny = 10, 5
display(datT[:ny*nx], nx, ny)

# 予測結果の出力
lab_pred = np.argmax(Yt[:ny*nx, :], axis=1)
lab_pred = lab_pred.reshape((ny, nx))
print('↑の画像を入力したときにモデルが予測するクラス')
print(lab_pred)
print()
print(f'テスト: {ce:.3f}  {count/NT:.3f}')

----
## 2クラスロジスティック回帰モデルの勾配の計算
----





「ロジスティック回帰＋勾配法によるパラメータの最適化 (3)」の notebook で説明している2クラス識別のロジスティック回帰モデルの勾配を自分で導出してみましょう．
notebookA の式$(5)$ を導くことが目標です．






---
### Step1



$s_n$ を

$$
s_n = w_0 + \sum_{d=1}^{D}w_dx_{n,d} \quad (*1)
$$

とおくと，notebookA の式$(1)$より，

$$
f(\mathbf{x}_n) = \sigma(s_n)
$$

と表せるので，式$(3)$を

$$
\ell_n = y_n\log \sigma(s_n) + (1-y_n)\log(1-\sigma(s_n))
$$

と書ける．
したがって，

$$
\begin{aligned}
\frac{\partial \ell_n}{\partial w_d} &= y_n \frac{\partial }{\partial w_d}\log \sigma(s_n) + (1-y_n) \frac{\partial }{\partial w_d} \log(1-\sigma(s_n)) \\
&= \frac{y_n}{\fbox{A}} \frac{\partial \sigma(s_n)}{\partial w_d} + \frac{1-y_n}{\fbox{B}} \cdot (-1) \cdot \frac{\partial \sigma(s_n)}{\partial w_d} \quad (*2)
\end{aligned}
$$

となる．

#### 問題1

式$(*2)$の $\fbox{A}, \fbox{B}$ に入る式を求めなさい．

---
### Step2


式$(*2)$の $\frac{\partial \sigma(s_n)}{\partial w_d}$ は

$$
\frac{\partial \sigma(s_n)}{\partial w_d} = \frac{d \sigma(s_n)}{d s_n} \frac{\partial s_n}{\partial w_d} \quad (*3)
$$

と変形できる．
このとき，

$$
\frac{d \sigma(s_n)}{d s_n} = \sigma(s_n)(1-\sigma(s_n)) \quad (*4)
$$

および

$$
\frac{\partial s_n}{\partial w_d} = x_{n,d} \quad (*5)
$$

となる（$x_{n,0} \equiv 1$）．

#### 問題2

1. $\sigma(s) = \frac{1}{1+e^{-s}}$ に対して $\frac{d \sigma(s)}{ds}$ を求めなさい（$s$の式で表しなさい）．
1. ↑で求めた式を $\sigma(s)$ のみ用いた式に変形することで，式$(*4)$ が成り立つことを示しなさい．

#### 問題3

式$(*5)$が成り立つことを示しなさい．

---
### Step3

式$(*2)$ に式$(*4), (*5)$ を代入して整理すると，，notebookA の式$(5)$ が得られる．



#### 問題4

上記のことを示しなさい．